In [1]:
import random
import numpy as np
import scipy.io.wavfile
import scipy.signal
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta, Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy #TODO what to import here?
from tensorflow.keras.regularizers import l1
from tensorflow.keras.backend import clear_session
import tensorflow as tf
from tensorflow.keras.layers import Dropout
from sklearn.ensemble import RandomForestClassifier

class DataKeeper:
    def __init__(self):
        self.RawDatas = []
        self.RawLabel = []
    def AddNewType(self, data, label):
        self.RawDatas.append(data)
        self.RawLabel.append(label)
    
    @property
    def KeptTypeCount(self):
        return len(self.RawDatas)
    
    def GetData(self, index):
        return self.RawDatas[index]
    
    def GenerateTrainTestValid(self, alpha=0.8):
        #if aplha is 0.8 -> train is 80%, valid is 10%, test is 10%
        SumRawDatas = []
        #létrehozzuk a teljes adatállományt egy tömbben
        for x in range(0, len(self.RawDatas)):
            for element in self.RawDatas[x]:
                SumRawDatas.append([element,self.RawLabel[x]])
        #összekeverjük
        random.shuffle(SumRawDatas)
        #szétválogatjuk 3 részre
        train = SumRawDatas[:int(len(SumRawDatas)*alpha)]
        test_valid = SumRawDatas[int(len(SumRawDatas)*alpha):]
        test = test_valid[:int(len(test_valid)*0.5)]
        valid = test_valid[int(len(test_valid)*0.5):]
        
        #x_train, y_train létrehozása
        self.x_train = []
        self.y_train = []
        for x in train:
            self.x_train.append(x[0])
            self.y_train.append(x[1])
            
        #x_test, y_test létrehozása
        self.x_test = []
        self.y_test = []
        for x in test:
            self.x_test.append(x[0])
            self.y_test.append(x[1])
            
        #x_valid, y_valid létrehozása
        self.x_valid = []
        self.y_valid = []
        for x in train:
            self.x_valid.append(x[0])
            self.y_valid.append(x[1])
            
        #numpy tömbbe konvertálás
        self.x_train = np.array(self.x_train)
        self.y_train = np.array(self.y_train)
        
        self.x_test = np.array(self.x_test)
        self.y_test = np.array(self.y_test)
        
        self.x_valid = np.array(self.x_valid)
        self.y_valid = np.array(self.y_valid) 
        print("Kész")

In [2]:
def WavToSplittedArray(filename="", window_size = 100):
    fs, wave = scipy.io.wavfile.read(filename) 
    windows_count = int(len(wave)/window_size)

    kimenet = []
    for i in range(windows_count):
      buffer = []
      for x in range(window_size):
        buffer.append(wave[i*window_size+x])
      kimenet.append(buffer)
    return kimenet

In [5]:
ablakmeret = 1500

Adatgyujto = DataKeeper()

Adatgyujto.AddNewType(WavToSplittedArray("sounds/1lehuzva_mono.wav", ablakmeret), 0)
Adatgyujto.AddNewType(WavToSplittedArray("sounds/2lehuzva_mono.wav", ablakmeret), 1)
Adatgyujto.AddNewType(WavToSplittedArray("sounds/alapjarat_mono.wav", ablakmeret),2)


In [6]:
Adatgyujto.GenerateTrainTestValid()

Kész


In [7]:
model = RandomForestClassifier(n_estimators=100)

model.fit(X=Adatgyujto.x_train, y=Adatgyujto.y_train)


RandomForestClassifier()

In [8]:
predikalando = 60

pred = model.predict(Adatgyujto.x_test[predikalando].reshape(1,ablakmeret))

print("ennek kene hogy legyen: " + str(Adatgyujto.y_test[predikalando]))
print(pred)

ennek kene hogy legyen: 2
[2]


In [9]:
import pickle
pkl_filename = "KerasModels/RandomForest/model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

FileNotFoundError: [Errno 2] No such file or directory: 'KerasModels/RandomForest/model.pkl'

In [10]:
model.score(Adatgyujto.x_test,Adatgyujto.y_test)

0.7332268370607029